In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from tensorflow.keras import layers, models
#from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#import the dataset
data = pd.read_csv("UNSW_NB15_training-set.csv")
data

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.090200,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.000300,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.005100,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.660800,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.002500,...,1,3,0,0,0,2,3,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,82328,0.000005,udp,-,INT,2,0,104,0,200000.005100,...,1,2,0,0,0,2,1,0,Normal,0
82328,82329,1.106101,tcp,-,FIN,20,8,18062,354,24.410067,...,1,1,0,0,0,3,2,0,Normal,0
82329,82330,0.000000,arp,-,INT,1,0,46,0,0.000000,...,1,1,0,0,0,1,1,1,Normal,0
82330,82331,0.000000,arp,-,INT,1,0,46,0,0.000000,...,1,1,0,0,0,1,1,1,Normal,0


In [ ]:
#find missing values in data
data.isnull().sum()

,0
id,0
dur,0
proto,0
service,0
state,0
spkts,0
dpkts,0
sbytes,0
dbytes,0
rate,0


In [ ]:
#check datatype of each column
data.dtypes

,0
id,int64
dur,float64
proto,object
service,object
state,object
spkts,int64
dpkts,int64
sbytes,int64
dbytes,int64
rate,float64


In [ ]:
# Data preprocessing function
def preprocess_data(data):
    data = data.drop(["id", "attack_cat"], axis=1, errors="ignore")  # Drop irrelevant columns
    label_encoder = LabelEncoder()
    for column in data.select_dtypes(include=["object"]).columns:
        data[column] = label_encoder.fit_transform(data[column])
    scaler = MinMaxScaler()
    data[data.columns] = scaler.fit_transform(data)
    return data

train_data = data
# test_data = pd.read_csv("UNSW_NB15_training-set.csv")

# Preprocess training data
X_train = preprocess_data(train_data.drop("label", axis=1))
y_train = train_data["label"]

# X_test = preprocess_data(test_data.drop("label", axis=1))
# y_test = test_data["label"]

In [ ]:
#perform PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=10)  # Specify the number of components
data_pca = pca.fit_transform(X_train)

print("Explained Variance Ratio:", pca.explained_variance_ratio_)
print("Components:", pca.components_)

Explained Variance Ratio: [0.57952407 0.12535638 0.08260934 0.03594802 0.02639051 0.02636018
 0.02334948 0.01894764 0.01400465 0.01307402]
Components: [[ 6.67892473e-03  1.00980835e-02  3.39738074e-02 -8.17946601e-02
   1.52701976e-03  1.59628543e-03  6.02274577e-04  9.34637395e-04
  -9.41086095e-02 -1.83743542e-01  4.09531065e-01 -1.36617942e-02
   2.40721693e-02  9.56124934e-04  1.18084535e-03 -7.76169551e-03
   1.69365983e-03  4.56720073e-03  1.22707214e-03  5.16233266e-01
   2.79571192e-01  2.76865274e-01  5.27863237e-01  1.66062246e-02
   1.02823790e-02  1.03391411e-02  2.51272883e-02  7.34254518e-02
   8.17181018e-04  3.24231715e-04 -1.11503233e-01 -9.53768901e-02
  -7.75388173e-02 -8.09774044e-02 -9.26842209e-02 -1.07745974e-01
   4.39470182e-03  4.43524160e-03  9.42489579e-03 -8.09272970e-02
  -1.15321009e-01 -8.80250555e-03]
 [ 6.98079401e-03  4.76863750e-02 -5.62594295e-02  7.51573889e-02
  -1.73945338e-03 -3.49184673e-03  2.91105243e-04 -2.69466295e-03
   5.28519164e-02  7.9

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=256, random_state=42)
rf_classifier.fit(data_pca, y_train)

RandomForestClassifier(n_estimators=256, random_state=42)

In [ ]:
test_data = pd.read_csv("UNSW_NB15_testing-set.csv")
X_test = preprocess_data(test_data.drop("label", axis=1))
y_test = test_data["label"]
X_test_encoded = pca.fit_transform(X_test)
y_pred = rf_classifier.predict(X_test_encoded)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.78      0.85     56000
           1       0.90      0.97      0.94    119341

    accuracy                           0.91    175341
   macro avg       0.92      0.88      0.89    175341
weighted avg       0.91      0.91      0.91    175341

